## 0. Preprocessing (same as HW2)

In [1]:
from scipy.io import arff
import pandas as pd
import numpy as np

In [2]:
data = arff.loadarff('4year.arff')
df = pd.DataFrame(data[0])

In [3]:
df['bankruptcy'] = (df['class']==b'1')
del df['class']
df.columns = ['X{0:02d}'.format(k) for k in range(1,65)] + ['bankruptcy']

In [4]:
df.describe()

,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,...,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64
count,9791.000000,9791.000000,9791.000000,9749.000000,9.771000e+03,9791.000000,9791.000000,9773.000000,9792.000000,9791.000000,...,9.792000e+03,9771.000000,9791.000000,9776.000000,9791.000000,9178.000000,9760.000000,9.771000e+03,9749.000000,9561.000000
mean,0.043019,0.596404,0.130959,8.136600,6.465164e+01,-0.059273,0.059446,19.884016,1.882296,0.389040,...,7.686330e+03,-0.992263,0.035022,1.133287,0.856053,118.156064,25.194430,2.015157e+03,8.660813,35.949619
std,0.359321,4.587122,4.559074,290.647281,1.475939e+04,6.812754,0.533344,698.697015,17.674650,4.590299,...,7.605261e+04,77.007971,8.945365,8.038201,26.393305,3230.316692,1099.260821,1.171461e+05,60.838202,483.318623
min,-12.458000,0.000000,-445.910000,-0.045319,-3.794600e+05,-486.820000,-12.458000,-1.848200,-0.032371,-445.910000,...,-7.132200e+05,-7522.100000,-597.420000,-30.892000,-284.380000,0.000000,-12.656000,-1.496500e+04,-0.024390,-0.000015
25%,0.001321,0.263145,0.020377,1.047000,-5.121700e+01,-0.000578,0.003004,0.428300,1.006675,0.294440,...,2.184000e+01,0.003121,0.008768,0.885722,0.000000,5.356325,4.267700,4.323400e+01,2.938800,2.012900
50%,0.041364,0.467740,0.199290,1.591800,-5.557600e-02,0.000000,0.048820,1.088700,1.161300,0.510450,...,9.503300e+02,0.043679,0.098026,0.958305,0.002129,9.482000,6.283550,7.472900e+01,4.848900,4.041600
75%,0.111130,0.689255,0.410670,2.880400,5.573200e+01,0.065322,0.126940,2.691000,1.970225,0.714290,...,4.694550e+03,0.117170,0.242680,0.996163,0.211790,19.506000,9.938200,1.233450e+02,8.363800,9.413500
max,20.482000,446.910000,22.769000,27146.000000,1.034100e+06,322.200000,38.618000,53209.000000,1704.800000,12.602000,...,6.123700e+06,112.020000,226.760000,668.750000,1661.000000,251570.000000,108000.000000,1.077900e+07,5662.400000,21153.000000


In [5]:
sum(df.bankruptcy == True)

515

In [6]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
X_imp = imp_mean.fit_transform(df.values)

In [7]:
from sklearn.model_selection import train_test_split

X, y = X_imp[:, :-1], X_imp[:, -1]

X_train, X_test, y_train, y_test =\
    train_test_split(X, y, 
                     test_size=0.3, 
                     random_state=0, 
                     stratify=y)

In [8]:
from sklearn.preprocessing import StandardScaler

stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

## 1. Extract 3 features using PCA method

In [9]:
from sklearn.decomposition import PCA

In [12]:
features_num = 3    #set the num of features 
pca = PCA(n_components=3)
X_train_pca = pca.fit_transform(X_train_std)
X_test_pca = pca.transform(X_test_std)

In [13]:
pca.explained_variance_ratio_

array([0.17496515, 0.09121706, 0.08114627])

In [14]:
X_train_pca

array([[-0.27866667, -0.12837331, -0.83950823],
       [-0.49169702, -0.0395489 , -0.37901699],
       [ 0.25337512, -0.0080635 ,  0.52861319],
       ...,
       [ 0.61759929, -0.15244715, -0.70213599],
       [-1.55606595,  0.21775369,  3.17680997],
       [ 0.46442696, -0.09411646, -0.460084  ]])

## 2. Using the selected features, apply LR / SVM / decision tree using pipeline.

In [15]:
#import packages
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline

### 2.1 LR

In [55]:
lr = LogisticRegression(random_state=1)
pipe_lr = make_pipeline(StandardScaler(), PCA(n_components=3), lr)
pipe_lr.fit(X_train, y_train)
y_pred = pipe_lr.predict(X_test)

#calc accuracy
print('#0. Logistic Regression')
print('The training accuracy is :', pipe_lr.score(X_train,y_train))
print('The test accuracy is :',pipe_lr.score(X_test, y_test))

#0. Logistic Regression
The training accuracy is : 0.946892325649256
The test accuracy is : 0.9462219196732471


### 2.2 SVM

In [25]:
svm = SVC(random_state=1)
pipe_svm = make_pipeline(StandardScaler(), PCA(n_components=3), svm)
pipe_svm.fit(X_train, y_train)
y_pred = pipe_svm.predict(X_test)

#calc accuracy
print('#1. SVM')
print('The training accuracy is :', pipe_svm.score(X_train,y_train))
print('The test accuracy is :',pipe_svm.score(X_test, y_test))

#1. SVM
The training accuracy is : 0.9479136270790779
The test accuracy is : 0.9472430224642614


### 2.3 Decision Tree

In [29]:
dt_clf = DecisionTreeClassifier(max_depth=5, random_state=1)
pipe_dt = make_pipeline(StandardScaler(), PCA(n_components=3), dt_clf)
pipe_dt.fit(X_train, y_train)
y_pred = pipe_dt.predict(X_test)

#calc accuracy
print('#2. Decision Tree')
print('The training accuracy is :', pipe_dt.score(X_train,y_train))
print('The test accuracy is :',pipe_dt.score(X_test, y_test))

#2. Decision Tree
The training accuracy is : 0.9496644295302014
The test accuracy is : 0.9469026548672567


## 3. Use grid search for finding optimal hyperparameters. (choose one method, 10-fold cv)

In [30]:
#import packages
from sklearn.model_selection import GridSearchCV

### Choose LR

In [69]:
penalty_list = ['l1','l2']
solver_list = ['liblinear', 'newton-cg', 'sag']
c_list = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
param_grid = {'logisticregression__penalty': penalty_list, 'logisticregression__solver': solver_list, 'logisticregression__C': c_list}
#There should be two underscores between estimator name and it's parameters in a Pipeline!

In [72]:
gs_lr = GridSearchCV(pipe_lr, param_grid, scoring='accuracy', cv=10, n_jobs=-1)
gs_lr = gs_lr.fit(X_train_pca, y_train)

In [75]:
print('#0. Logistic Regression')
print('The highest accuracy is: ', gs_lr.best_score_)
print('The best parameters are: \n', gs_lr.best_params_)

#0. Logistic Regression
The highest accuracy is:  0.9474758996403567
The best parameters are: 
 {'logisticregression__C': 0.001, 'logisticregression__penalty': 'l1', 'logisticregression__solver': 'liblinear'}
